<a href="https://colab.research.google.com/github/3lueLightning/tutorials/blob/main/rag_development_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install -q langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00


In [34]:
MOVIES_URL = "https://www.empireonline.com/movies/features/best-movies-2/"

In [21]:
from langchain_community.document_loaders import WebBaseLoader

In [32]:
import os

os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"


In [36]:
tt = WebBaseLoader(MOVIES_URL).load()
tt[0].page_content[:200]

In [42]:
from bs4 import SoupStrainer

movie_scraper = WebBaseLoader(
    MOVIES_URL,
    bs_kwargs = {
        "parse_only": SoupStrainer("div", class_="listicleItem_listicle-item__content__Lxn1Y")
    }
)

In [43]:
movie_reviews_raw = movie_scraper.load()

In [101]:
from langchain.docstore.document import Document

def split_movies(page):
  movie_reviews = page.page_content.strip().split("\n\n")
  movie_reviews = [reviews.split("\n")[0].strip() for reviews in movie_reviews]
  movie_docs = [Document(review, metadata=page.metadata) for review in movie_reviews]
  return movie_docs

In [102]:
movie_reviews = split_movies(movie_reviews_raw[0])
movie_reviews[0]

Document(page_content="Making his uber cool and supremely confident directorial debut, Quentin Tarantino hit audiences with a terrific twist on the heist-gone-wrong thriller. For the most part a single location chamber piece, Reservoir Dogs delights in ricocheting the zing and fizz of its dialogue around its gloriously —and indeed gore-iously) — intense setting, with the majority of the movie's action centring around one long and incredibly bloody death scene. Packing killer lines, killer needledrops, and killer, er, killers too, not only is this a rollicking ride in its own right, but it also set the blueprint for everything we've come to expect from a Tarantino joint. Oh, and by the way: Nice Guy Eddie was shot by Mr. White. Who fired twice. Case closed.", metadata={'source': 'https://www.empireonline.com/movies/features/best-movies-2/'})